# Analyze Data
- t-SNE/PCA embedding clustering
- Object detection visualization

In [ ]:
import json

from os import listdir, path

from utils.cluster_utils import pca_kmeans, tsne_kmeans
from utils.cluster_utils import plot_clusters, visualize_pca_clusters, visualize_tsne_clusters

from params.collections import MUSEUMS
from Museum import Museum

## t-SNE/PCA: by museum

In [ ]:
museum_info = MUSEUMS["brasiliana"]
Museum.prep_dirs(museum_info)

In [ ]:
embeding_files = sorted([f for f in listdir(Museum.DIRS["embeddings"]) if f.endswith(".json")])

raw_embs = []
image_paths = []

for idx, io_file in enumerate(embeding_files):
  qid = io_file.replace(".json", "")
  img_file = io_file.replace(".json", ".jpg")

  embeding_path = path.join(Museum.DIRS["embeddings"], io_file)
  with open(embeding_path, "r", encoding="utf8") as f:
    m_embs = json.load(f)

  raw_embs.append(m_embs[qid]["siglip2"])
  image_paths.append(path.join(Museum.IMGS["500"], img_file))

In [ ]:
pca_vals, pca_clusters, pca_centers = pca_kmeans(raw_embs, n_clusters=4)
plot_clusters(pca_clusters, pca_vals, title="clip pca")

In [ ]:
visualize_pca_clusters(raw_embs, image_paths, n_clusters=4, grid_dim=6)

In [ ]:
tsne3_vals, tsne3_clusters, tsne3_centers = tsne_kmeans(raw_embs, n_clusters=4, n_components=3)
plot_clusters(tsne3_clusters, tsne3_vals, title="clip tsne 3D")

In [ ]:
tsne2_vals, tsne2_clusters, tsne2_centers = tsne_kmeans(raw_embs, n_clusters=4, n_components=2)
plot_clusters(tsne2_clusters, tsne2_vals, title="clip tsne 2D")

In [ ]:
visualize_tsne_clusters(raw_embs, image_paths, n_clusters=4, grid_dim=6)

## Visualize Objects

In [ ]:
import json

from os import listdir, path

from PIL import Image as PImage, ImageOps as PImageOps, ImageDraw as PImageDraw

from params.collections import MUSEUMS
from Museum import Museum

### Visualize Boxes

In [ ]:
museum_info = MUSEUMS["brasiliana"]
Museum.prep_dirs(museum_info)

In [ ]:
obj_files = sorted([f for f in listdir(Museum.DIRS["objects"]) if f.endswith(".json")])

In [ ]:
for fname in obj_files:
  with open(path.join(Museum.DIRS["objects"], fname), "r") as inp:
    iboxes = json.load(inp)

  if len(iboxes) < 1:
    continue

  image_file_path = path.join(Museum.IMGS["900"], fname.replace(".json", ".jpg"))
  image = PImageOps.exif_transpose(PImage.open(image_file_path).convert("RGB"))
  iw,ih = image.size
  draw = PImageDraw.Draw(image)

  for box in iboxes:
    label, (x0,y0,x1,y1) = box["label"], box["box"]
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)

  display(image)

### Create Mosaic from id + object indexes

In [ ]:
import json

import numpy as np

from os import path
from PIL import Image as PImage

In [ ]:
idObjIdxs_data = [
  {"id":"Q42713599","objIdxs":[0]},
  {"id":"Q42713701","objIdxs":[0,1]},
  {"id":"Q52301360","objIdxs":[0,1]},
  {"id":"Q52303067","objIdxs":[0]},
  {"id":"Q52303343","objIdxs":[0,1]},
  {"id":"Q52303884","objIdxs":[0]},
  {"id":"Q59924902","objIdxs":[0,1,2,3,4,5]},
  {"id":"Q59924903","objIdxs":[0]},
  {"id":"Q59925006","objIdxs":[0]},
  {"id":"Q59925012","objIdxs":[0,1,2,3]},
  {"id":"Q59925013","objIdxs":[0,1,2]},
  {"id":"Q59925016","objIdxs":[0,1]},
  {"id":"Q59925022","objIdxs":[0]},
  {"id":"Q59925023","objIdxs":[0,1]},
  {"id":"Q59925024","objIdxs":[0,1,2,4,5,6,7]},
  {"id":"Q59925026","objIdxs":[0,1,2,3]},
  {"id":"Q59925027","objIdxs":[1]},
  {"id":"Q59925028","objIdxs":[0]},
  {"id":"Q59925033","objIdxs":[0,1]},
  {"id":"Q59925682","objIdxs":[0]},
  {"id":"Q59925704","objIdxs":[1]},
  {"id":"Q59925705","objIdxs":[0]},
  {"id":"Q59925718","objIdxs":[0]},
  {"id":"Q59954329","objIdxs":[0]},
  {"id":"Q123784523","objIdxs":[]},
  {"id":"Q124620698","objIdxs":[]},
  {"id":"Q125101557","objIdxs":[]},
  {"id":"Q59954335","objIdxs":[0]},
  {"id":"Q59954338","objIdxs":[0,1,2]},
  {"id":"Q59954339","objIdxs":[0]},
  {"id":"Q59954346","objIdxs":[0,1]}
]

DATA_FILE = "./metadata/json/20250619_processed.json"
IMG_DIR = "../../imgs/arts/full"

with open(DATA_FILE, "r") as ifp:
  all_data = json.load(ifp)

In [ ]:
height_min = 1e6
sizes = {}

for idObjIdxs in idObjIdxs_data:
  if len(idObjIdxs["objIdxs"]) < 1:
    continue
  id = idObjIdxs["id"]
  img = PImage.open(path.join(IMG_DIR, f"{id}.jpg"))
  iw,ih = img.size
  sizes[id] = (iw,ih)
  for idx in idObjIdxs["objIdxs"]:
    (x0,y0,x1,y1) = all_data[id]["objects"][idx]["box"]
    crop_h = ih * (y1 - y0)
    height_min = min(height_min, crop_h)

print(height_min)

In [ ]:
width_sum = 0

for idObjIdxs in idObjIdxs_data:
  if len(idObjIdxs["objIdxs"]) < 1:
    continue
  id = idObjIdxs["id"]
  iw,ih = sizes[id]
  for idx in idObjIdxs["objIdxs"]:
    (x0,y0,x1,y1) = all_data[id]["objects"][idx]["box"]
    crop_w = iw * (x1 - x0)
    crop_h = ih * (y1 - y0)
    width_sum += (height_min / crop_h) * crop_w

mos_w = int((width_sum * height_min) ** 0.5)
mos_h = int(1.2 * mos_w)
print(mos_w, mos_h)

In [ ]:
mos_img = PImage.fromarray(np.zeros((mos_h, mos_w))).convert("RGB")

cur_x, cur_y = 0, 0

for idObjIdxs in idObjIdxs_data:
  if len(idObjIdxs["objIdxs"]) < 1:
    continue
  id = idObjIdxs["id"]
  img = PImage.open(path.join(IMG_DIR, f"{id}.jpg"))
  iw,ih = img.size
  for idx in idObjIdxs["objIdxs"]:
    (x0,y0,x1,y1) = all_data[id]["objects"][idx]["box"]
    crop_w = iw * (x1 - x0)
    crop_h = ih * (y1 - y0)

    scale_factor = height_min / crop_h
    crop_w, crop_h = int(scale_factor * crop_w), int(scale_factor * crop_h)

    crop_img = img.crop((int(x0 * iw), int(y0 * ih), int(x1 * iw), int(y1 * ih))).resize((crop_w, crop_h))

    if cur_y >= mos_h:
      print("break")
      break

    mos_img.paste(crop_img, (cur_x, cur_y))
    cur_x += crop_w

    if cur_x > mos_w:
      overflow_x = cur_x - mos_w
      crop_img = crop_img.crop((crop_w - overflow_x, 0, crop_w, crop_h))
      cur_x = 0
      cur_y += crop_h
      mos_img.paste(crop_img, (cur_x, cur_y))
      cur_x += overflow_x

if cur_x < mos_w and cur_y < mos_h:
  empty_w = mos_w - cur_x
  row = mos_img.crop((0, 0, empty_w, height_min))
  mos_img.paste(row, (cur_x, cur_y))

mos_img = mos_img.crop((0, 0, mos_w, cur_y + crop_h))
mos_img.thumbnail((1024,1024))

print(mos_img.size)
display(mos_img)

## Describe Object Crops

In [ ]:
import json

from utils.data_utils import Describer

PREFIX = "20250705"
DATA_DIR = "./metadata/json"
DATA_FILE = f"{DATA_DIR}/{PREFIX}_processed.json"
IMG_DIR = "../../imgs/arts/full"

with open(DATA_FILE, "r", encoding="utf-8") as ifp:
  all_data = json.load(ifp)

mDescriber = Describer(PREFIX, IMG_DIR)

In [ ]:
obj_boxes = []
obj_label = "palm tree"

for k,v in all_data.items():
  for odata in [o for o in v["objects"] if o["label"] == obj_label]:
    obj_boxes.append({"id": k} | odata)

sorted_boxes = sorted(obj_boxes, key=lambda x:x["score"], reverse=True)
sorted_box_keys = [(bo["id"], *bo["box"]) for bo in sorted_boxes]
len(sorted_box_keys)

In [ ]:
descriptions = mDescriber.describe(sorted_box_keys, num_images=16, words_offset=0)
descriptions